# Идея проекта, выборка и предобработка данных
<hr>

## Введение

```Текущая accuracy: 60-70%```

Проект представляет из себя анализатор дипфейков на основе анализа артефактов в различных зонах лица

В ходе предварительного анализа предметной области было выделено 3 наиболее перспективных для реализации метода анализа дипфейков:

- анализ движений глаз (саккад)
- анализ синзронизации речи и движений губ
- анализ артефактов

Так как анализ движений возможен только в динамике и довольно сложен в реализации, было решено реализовать метод анализа артефактов на статических изображениях (фото)

- В качестве "источника вдохновения" использовался [проект](https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb), доступный по ссылке: 

https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb

- В качестве датасета использовались фрагменты датасета [Deep Fake Detection Challenge DFDC](https://www.kaggle.com/competitions/deepfake-detection-challenge):

https://www.kaggle.com/competitions/deepfake-detection-challenge

- Про [распознанвание зон лица](https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/) можно прочесть по ссылке:

https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/

- Сжатый [датасет 300 Faces In-the-Wild Challenge (300-W)](https://ibug.doc.ic.ac.uk/resources/300-W/) для распознавания точек на лице и аннотации к нему:

https://ibug.doc.ic.ac.uk/resources/300-W/

## Пайплайн проекта

Проект работает в следующем порядке:

- datacollector.py, faceparts.py:
1. Считывание по 1 кадру из видео-датасета DFDC (5 x 10 GB, 5 x 1700 видео)
2. Распознавание 68 точек на лице с помощью OpenCV, DLib и датасета 300-W
3. Отсев фото, которые не удалось распознать (остается около 3500 фото), запись *размера* лица и бинарного значения яркости *пикселей* в 68 зонах вокруг ключевых точек в их черно-белой версии (размер зон - 8-17% от ширины лица, сейчас подбирается) в dataset (5 х 0.2-0.5 GB)
> псевдокод: ```image.to_greyscale().cut([zone_schape])```

- notebook.ipynb (до метки "Legacy"):
1. Добавление гистограмм распределения яркости к склейке датасетов
2. Добавление метрик std и noise для каждой зоны
3. Очистка от исходных массивов пикселей, запись на диск (30 MB)

- notebook2.ipynb:
1. Отсекание лишних фейков из исходного датасета (остается 1000=500*2 из 3500 фото) для избавления от перекоса (по крайней мере в процессе подбора методов, такой датасет удобнее, иначе недо/переобучение вощникает слишком часто)
2. Сбор std и avg значений для каждого из классов => определение точек, с наибольшим значением ```|avg_fake-avg_real| / (std_fake + std_real)```
> sklearn.feature_selection.SelectKBest работает плохо, т.к. не включает метрики для всего лица, которые нужны для нормализации значений по точкам

3. Выборка самых показательных features + features по всему лицу
4. Обучение моделей

> Так как 5 из 8 моделей дают сопоставимую точность, на данном этапе целесообразно работать надо подбором фичей и размерами сканируемых зон, мб фильтрацией плохих/маленьких фото, тюнинг моделей (тщательный) следует проводть позднее

## Потенциальные темы НИРов

> 1 тема != 1 НИР, темы могут быть объединены или разбиты

- процесс сборки данных: введение про проблему и методы анализа дипфейков, датасет DFDC, распознавание точек, OpenCV, DLib, датасет 300-W
- Предобработка данных: фильтрация, гистограммы и агрегационные метрики
- Подбор features, метрики метрик, метрики по всему лицу
- Выбор и тюнинг моделей, влияние размера и перекоса датасета и т.д.

На более поздних этапах (опционально):
- выстраивание вокруг модели эконсистемы: веб-приложение, комплект клиентов на React-native / чат-бот, Google OAuth, Metamask Auth
- Биллинг для приложений, ценообразование, токены (можно крипту из тестнета (Goerli, Ether testnet) ради забавы прикрутить), мониторинг (Grafana?), Docker-Kuber, балансировка и прочие умные штуки

# Обработка данных
<hr>

> 990 фото, баланс классов 1:1, размер зон - 8..17% от ширины лица (в процессе экспериментов), f1-метрика на разных моделях - 0.60..0.70

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [1]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [2]:
import pandas as pd
import numpy as np

In [31]:
df = pd.read_json("../deep_fake_src/dfdc_dataframes/df_total_0_4_compact_frame_10.json")

In [32]:
# Балансировка классов 1:1
# Инверсные метрики для полиномов

# df = df.drop(columns=['index','face_size_px'])
df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / x)

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

print(df.shape)
# df = df.reset_index()
df.head()

(990, 281)


,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
2077,69,ilkzwzcrnh.mp4,True,36100,15.255370,8.128830,7.127124,10.231011,6.934684,12.679608,...,9.375000,10.937500,6.250000,17.187500,0.368421,0.000028,0.042086,0.003911,5.231884,2.714286
2627,8,eutaedkkmc.mp4,True,44100,42.581615,38.053073,23.962713,28.586039,25.987966,42.194657,...,33.000000,58.000000,67.000000,67.000000,0.569161,0.000023,0.014073,0.010035,2.672727,1.756972
253,265,iodwjkqblp.mp4,True,85264,13.821499,24.114428,20.294887,16.356696,13.972331,5.977404,...,0.510204,0.000000,0.510204,2.551020,0.283824,0.000012,0.019291,0.001882,4.284623,3.523306
1763,967,wxukovbmac.mp4,True,86436,8.919738,21.820295,18.735458,19.286724,13.529088,6.010966,...,7.653061,8.673469,9.183673,9.693878,0.284604,0.000012,0.021165,0.002600,5.206988,3.513659
718,240,kmzlyhldkp.mp4,True,341056,13.699687,15.442610,9.899472,21.433623,27.314534,32.562686,...,0.637755,2.423469,1.913265,3.188776,0.066265,0.000003,0.025027,0.000704,25.263407,15.090973


In [33]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
True,100471.854545,46.581447,539.532254,0.179087,0.289892,0.000015,0.023683,0.003183,10.237802,5.425916


In [34]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
False,92126.343434,49.182392,534.771606,0.200743,0.308146,0.000016,0.022526,0.003057,8.383215,4.79785


In [35]:
# самые ярко-различающиеся по классам нfeatures

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
pt_6_hist_noise_0.5,0.688294,0.972680,0.171216,0.284386
index,541.650505,406.274747,0.142813,135.375758
pt_24_hist_noise_0.5,1.649462,1.245546,0.139522,0.403916
pt_11_hist_noise_0.5,1.231487,1.606494,0.132139,0.375008
pt_30_hist_noise_0.5,1.692450,1.298086,0.131871,0.394364
pt_5_hist_noise_0.5,1.686957,2.192505,0.130314,0.505548
pt_19_hist_noise_0.5,1.641023,1.273310,0.126174,0.367713
pt_57_hist_noise_3,16.263845,12.640072,0.125373,3.623773
pt_54_hist_noise_0.5,1.306997,1.671118,0.122266,0.364121
pt_20_hist_noise_0.5,1.829038,1.431246,0.122011,0.397792


In [36]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
face_size_px_reversed,0.000016,0.000015,0.000031
overall_face_hist_std_reversed,0.003057,0.003183,0.006240
overall_face_std_reversed,0.022526,0.023683,0.046209
overall_face_hist_noise_0.5,0.200743,0.179087,0.379830
overall_face_hist_noise_3,0.308146,0.289892,0.598038
pt_8_hist_noise_0.5,0.507715,0.562259,1.069973
pt_7_hist_noise_0.5,0.540935,0.580272,1.121207
pt_9_hist_noise_0.5,0.687613,0.703464,1.391078
pt_57_hist_noise_0.5,0.871193,0.684589,1.555782
pt_10_hist_noise_0.5,0.715615,0.883977,1.599592


In [37]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
pt_6_hist_noise_0.5,0.284386,1.660974,0.171216
index,135.375758,947.925253,0.142813
pt_24_hist_noise_0.5,0.403916,2.895008,0.139522
pt_11_hist_noise_0.5,0.375008,2.837981,0.132139
pt_30_hist_noise_0.5,0.394364,2.990537,0.131871
pt_5_hist_noise_0.5,0.505548,3.879461,0.130314
pt_19_hist_noise_0.5,0.367713,2.914333,0.126174
pt_57_hist_noise_3,3.623773,28.903917,0.125373
pt_54_hist_noise_0.5,0.364121,2.978115,0.122266
pt_20_hist_noise_0.5,0.397792,3.260284,0.122011


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [38]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(7).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(7).index) +
    list(df_smart.filter(like='std', axis=0).head(5).index) +
    list(df_smart.filter(like='reversed', axis=0).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

print(len(features))
features

27


['pt_24_std',
 'pt_57_hist_std',
 'pt_58_hist_noise_3',
 'pt_57_hist_noise_3',
 'pt_40_hist_noise_3',
 'pt_57_std',
 'pt_45_hist_noise_3',
 'pt_11_hist_noise_0.5',
 'pt_39_hist_noise_3',
 'pt_39_std',
 'pt_54_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'pt_56_hist_noise_3',
 'pt_19_std',
 'pt_3_hist_noise_3',
 'overall_face_hist_noise_0.5',
 'overall_face_hist_std_reversed',
 'face_size_px_reversed',
 'overall_face_hist_noise_3_reversed',
 'pt_30_hist_noise_0.5',
 'overall_face_hist_noise_0.5_reversed',
 'pt_6_hist_noise_0.5',
 'overall_face_std_reversed',
 'pt_19_hist_noise_0.5',
 'pt_5_hist_noise_0.5',
 'overall_face_hist_std',
 'pt_24_hist_noise_0.5']

## Разбивка данных на train и test сплиты

In [39]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно?)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
2077,69,True,36100,15.255370,8.128830,7.127124,10.231011,6.934684,12.679608,17.449229,...,9.375000,10.937500,6.250000,17.187500,0.368421,0.000028,0.042086,0.003911,5.231884,2.714286
2627,8,True,44100,42.581615,38.053073,23.962713,28.586039,25.987966,42.194657,43.138841,...,33.000000,58.000000,67.000000,67.000000,0.569161,0.000023,0.014073,0.010035,2.672727,1.756972
253,265,True,85264,13.821499,24.114428,20.294887,16.356696,13.972331,5.977404,2.273473,...,0.510204,0.000000,0.510204,2.551020,0.283824,0.000012,0.019291,0.001882,4.284623,3.523306
1763,967,True,86436,8.919738,21.820295,18.735458,19.286724,13.529088,6.010966,5.874049,...,7.653061,8.673469,9.183673,9.693878,0.284604,0.000012,0.021165,0.002600,5.206988,3.513659
718,240,True,341056,13.699687,15.442610,9.899472,21.433623,27.314534,32.562686,22.342826,...,0.637755,2.423469,1.913265,3.188776,0.066265,0.000003,0.025027,0.000704,25.263407,15.090973


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [41]:
y = df_compact['fake']
X = pd.DataFrame(df_compact.drop(columns=['fake']))
# selector = SelectKBest(f_classif, k=7)
# X = selector.fit_transform(X, y)
X =  df_compact[features]

# mask = selector.get_support()
# X = X[X.columns[mask]]

# df_compact.filter(like='size', axis=0).head()

# print(df_compact[X.columns[mask]].shape)
# df_compact[X.columns[mask]].head()

pd.DataFrame(X).head()

,pt_24_std,pt_57_hist_std,pt_58_hist_noise_3,pt_57_hist_noise_3,pt_40_hist_noise_3,pt_57_std,pt_45_hist_noise_3,pt_11_hist_noise_0.5,pt_39_hist_noise_3,pt_39_std,...,face_size_px_reversed,overall_face_hist_noise_3_reversed,pt_30_hist_noise_0.5,overall_face_hist_noise_0.5_reversed,pt_6_hist_noise_0.5,overall_face_std_reversed,pt_19_hist_noise_0.5,pt_5_hist_noise_0.5,overall_face_hist_std,pt_24_hist_noise_0.5
2077,10.215440,1.096871,3.125000,1.562500,26.562500,4.681142,18.750000,0.000000,17.187500,9.354979,...,0.000028,2.714286,0.000000,5.231884,0.000000,0.042086,0.000000,0.0,255.719236,0.000000
2627,36.616143,0.974153,49.000000,45.000000,46.000000,32.821753,57.000000,0.000000,35.000000,31.690458,...,0.000023,1.756972,0.000000,2.672727,0.000000,0.014073,0.000000,0.0,99.655029,0.000000
253,28.050070,3.573960,8.163265,5.612245,35.204082,5.145456,5.612245,0.000000,19.387755,14.367133,...,0.000012,3.523306,0.000000,4.284623,0.000000,0.019291,0.000000,0.0,531.274400,0.000000
1763,14.850171,2.979053,4.081633,6.122449,16.836735,5.489436,15.816327,0.000000,12.755102,10.125672,...,0.000012,3.513659,0.000000,5.206988,0.000000,0.021165,0.000000,0.0,384.563282,0.000000
718,16.738826,9.002387,4.081633,3.826531,2.933673,9.113226,13.137755,0.382653,5.229592,9.812630,...,0.000003,15.090973,7.397959,25.263407,0.127551,0.025027,0.892857,0.0,1421.096479,3.443878


In [42]:
# print(df_compact.filter(like='size', axis=1).shape)
# df_compact.filter(like='size', axis=1).head()

In [43]:
# sizes_col = X.filter(like='size', axis=1).copy()
# X = pd.concat([
#     df_compact[X.columns[mask]],
#     sizes_col
# ], axis=0)
# # X.join(df_compact.filter(like='size', axis=1), lsuffix='_caller', rsuffix='_other')
# X.head()

In [44]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=420)

In [45]:
pd.DataFrame(X_train).head()

,pt_24_std,pt_57_hist_std,pt_58_hist_noise_3,pt_57_hist_noise_3,pt_40_hist_noise_3,pt_57_std,pt_45_hist_noise_3,pt_11_hist_noise_0.5,pt_39_hist_noise_3,pt_39_std,...,face_size_px_reversed,overall_face_hist_noise_3_reversed,pt_30_hist_noise_0.5,overall_face_hist_noise_0.5_reversed,pt_6_hist_noise_0.5,overall_face_std_reversed,pt_19_hist_noise_0.5,pt_5_hist_noise_0.5,overall_face_hist_std,pt_24_hist_noise_0.5
1874,27.099132,5.630900,3.750000,3.000000,18.750000,6.773839,16.500000,4.750000,13.000000,16.051814,...,0.000005,9.913474,0.500000,16.668673,0.000000,0.017269,3.250000,2.250000,1101.052972,2.750000
340,23.337885,3.844607,8.163265,2.551020,25.000000,3.624112,12.244898,0.000000,14.795918,13.382948,...,0.000011,6.244324,0.000000,13.393623,0.000000,0.028747,0.000000,0.000000,514.606183,0.000000
565,22.311417,2.506194,20.408163,12.244898,16.326531,9.820574,5.102041,0.000000,35.204082,26.446474,...,0.000011,4.587225,0.000000,7.753628,0.000000,0.018529,0.000000,0.000000,481.689734,0.000000
2811,10.632710,4.277456,14.506173,11.419753,34.259259,10.292824,23.148148,5.864198,24.382716,21.608780,...,0.000007,8.478228,2.469136,13.953750,0.308642,0.025473,4.320988,5.864198,766.006706,3.395062
2329,10.069751,2.926175,25.000000,13.281250,25.390625,12.582340,12.500000,6.250000,26.562500,24.731900,...,0.000009,4.482667,5.078125,6.560000,3.906250,0.014984,7.031250,6.250000,338.385216,1.562500


In [46]:
y_train.head()

1874    False
340      True
565      True
2811     True
2329    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [47]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier


In [62]:
import joblib

In [60]:
def save_model(model, X_train, y_train, X_test, y_test, p=2, name='my_model.pkl'):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    joblib.dump(clf, name)

In [48]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [49]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### SGDClassifier (f1=0.60) [to be modified]

In [50]:
print_polynomial_pipeline(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[72 29]
 [48 49]]
               precision    recall  f1-score   support

 class "real"       0.60      0.71      0.65       101
class "fakes"       0.63      0.51      0.56        97

     accuracy                           0.61       198
    macro avg       0.61      0.61      0.61       198
 weighted avg       0.61      0.61      0.61       198



### SVC (f1=0.64) [to be modified]

In [51]:
print_polynomial_pipeline(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[67 34]
 [36 61]]
               precision    recall  f1-score   support

 class "real"       0.65      0.66      0.66       101
class "fakes"       0.64      0.63      0.64        97

     accuracy                           0.65       198
    macro avg       0.65      0.65      0.65       198
 weighted avg       0.65      0.65      0.65       198



### LogisticRegression (f1=0.61) [to be modified]

In [52]:
print_polynomial_pipeline(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 

[[62 39]
 [32 65]]
               precision    recall  f1-score   support

 class "real"       0.66      0.61      0.64       101
class "fakes"       0.62      0.67      0.65        97

     accuracy                           0.64       198
    macro avg       0.64      0.64      0.64       198
 weighted avg       0.64      0.64      0.64       198



### NN - MLPClassifier (f1=0.66) [to be modified]

In [53]:
print_polynomial_pipeline(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(34,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
) 

[[59 42]
 [30 67]]
               precision    recall  f1-score   support

 class "real"       0.66      0.58      0.62       101
class "fakes"       0.61      0.69      0.65        97

     accuracy                           0.64       198
    macro avg       0.64      0.64      0.64       198
 weighted avg       0.64      0.64      0.64       198



### RandomForestClassifier (good, f1=0.64) [to be modified] 

In [63]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="random_forest_10.pkl"
) 

[[62 39]
 [22 75]]
               precision    recall  f1-score   support

 class "real"       0.74      0.61      0.67       101
class "fakes"       0.66      0.77      0.71        97

     accuracy                           0.69       198
    macro avg       0.70      0.69      0.69       198
 weighted avg       0.70      0.69      0.69       198



### DecisionTreeClassifier (f1=0.59) [to be modified]

In [55]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[44 57]
 [25 72]]
               precision    recall  f1-score   support

 class "real"       0.64      0.44      0.52       101
class "fakes"       0.56      0.74      0.64        97

     accuracy                           0.59       198
    macro avg       0.60      0.59      0.58       198
 weighted avg       0.60      0.59      0.58       198



### RadiusNeighborsClassifier (f1=0.46) [уязвим к выбросам]

In [56]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=100, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[35 66]
 [23 74]]
               precision    recall  f1-score   support

 class "real"       0.60      0.35      0.44       101
class "fakes"       0.53      0.76      0.62        97

     accuracy                           0.55       198
    macro avg       0.57      0.55      0.53       198
 weighted avg       0.57      0.55      0.53       198



### KNeighborsClassifier (f1=0.69) [to be modified]

In [57]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=6, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)

[[63 38]
 [31 66]]
               precision    recall  f1-score   support

 class "real"       0.67      0.62      0.65       101
class "fakes"       0.63      0.68      0.66        97

     accuracy                           0.65       198
    macro avg       0.65      0.65      0.65       198
 weighted avg       0.65      0.65      0.65       198

